<a href="https://colab.research.google.com/github/vladgap/Various/blob/ver1/MLN-051121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [14]:
import numpy as np

# Single Neuron

In [15]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ):
        """Overrides the 'linear' activation function."""
        self.activ = activ

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return -x

In [16]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
# neu.set_activ('sigmoid')
neu.run([1,1])

5.0

## AND gate

In [17]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


In [18]:
#proverka
# a=[]
# a.append(Perceptron(inputs=2))
# a[1].weights

## OR gate

In [19]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [20]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [61]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ):
        """Set the activation function to each neuron."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ) # set_activ for each neuron

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # STEP 3: Calculate the OUTPUT error terms
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the linear function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = (y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # STEP 3: Calculate the OUTPUT error terms
        self.d[-1] = (error) 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = fwd_error 
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

## XOR gate=(OR+NAND)+AND

In [22]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10 -10  15] sigmoid
Layer 1 Neuron 1 : [ 15  15 -10] sigmoid
Layer 2 Neuron 0 : [ 10  10 -15] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


In [23]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


# Training 

In [67]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(5000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

0.28165642201661084
0.26145569673726543
0.2477766559880984
0.1481717649259221
0.029030756522152066
0.011659416205884862
0.006873371617609248
0.004773601067881367
0.0036209558881454265
0.0029007032205328486
0.0024111543219050807
0.0020582308444621125
0.001792495102957434
0.0015856119905132587
0.0014202312993818373
0.0012851613151408877
0.0011728725788173943
0.0010781205110223207
0.000997144020337199
0.000927178130819963
0.0008661462467620938
0.0008124594771557755
0.0007648820586535608
0.0007224388627205007
0.0006843504321196428

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-5.51881509  5.71119023  2.77039168] sigmoid
Layer 1 Neuron 1 : [-5.8839065   5.74779262 -3.13391033] sigmoid
Layer 2 Neuron 0 : [-8.51919033  8.81752762  4.01340914] sigmoid

XOR Gate:
0 0 = 0.0256891907
0 1 = 0.9761472216
1 0 = 0.9707397731
1 1 = 0.0226578008


In [65]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[4.9,-0.9,2.1,0.2]]])
# mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(1000):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],[5.764286995])
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],[5.84646758])
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],[0.808633075])
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],[3.850298589])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()


Training Neural Network...

9.78075555265886
1.7172390510443396e-09
3.692766486071648e-17
7.936884165677096e-25
2.3665827156630354e-30

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253295e-05] linear

